In [3]:
%load_ext autoreload
%autoreload 2

import glob
from pathlib import Path
import multiprocessing as mp
import os
# fmt: off
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from datasets.preprocessing.scannet_preprocessing import ScannetPreprocessing

scannet_processor = ScannetPreprocessing(
    data_dir="/newfoundland/ScanNet", 
    save_dir=str(Path(module_path) / "data/processed/scannet"), 
    git_repo=str(Path(module_path) / "third_party/ScanNet"), 
    scannet200='false'
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
label database existed; not creating a new one. /home/ruizhu/Documents/Projects/Mask3D/data/processed/scannet/label_database.yaml


In [4]:
import re
'''
looks like 1 scenes by 3 scans: 
    - scene0055_00_vh_clean_2.ply
    - scene0055_01_vh_clean_2.ply
    - scene0055_02_vh_clean_2.ply
'''
_idx = 100
filepath = scannet_processor.files['validation'][_idx]
print(len(scannet_processor.files['validation']), filepath)

# scene, sub_scene = ScannetPreprocessing._parse_scene_subscene(scannet_processor.files['validation'][0].name)

scene_match = re.match(r"scene(\d{4})_(\d{2})", filepath.name)
scene, sub_scene = int(scene_match.group(1)), int(scene_match.group(2))
filebase = {
    "filepath": filepath,
    "scene": scene,
    "sub_scene": sub_scene,
    "raw_filepath": str(filepath),
    "file_len": -1,
}

print(scene, sub_scene)


312 /newfoundland/ScanNet/scans/scene0329_01/scene0329_01_vh_clean_2.ply
329 1


In [5]:
from utils.point_cloud_utils import load_ply_with_normals
import numpy as np

print(filepath)
coords, features, _ = load_ply_with_normals(filepath)

print(coords.shape, np.amax(coords, axis=0), np.amin(coords, axis=0))
print(features.shape, np.amax(features, axis=0), np.amin(features, axis=0)) # 3 channels features (RGB) + 3 channel normals
print(np.linalg.norm(features[:, 3:], axis=1)) # check if normals are unit vectors -> YES

file_len = len(coords)
filebase["file_len"] = file_len
points = np.hstack((coords, features))

print(points.shape) # (N, 9): x, y, z, R(255), G(255), B(255), nx, ny, nz

/newfoundland/ScanNet/scans/scene0329_01/scene0329_01_vh_clean_2.ply
(368473, 3) [ 9.308865 10.587272  2.998444] [ 0.2837082   0.9186443  -0.00797399]
(368473, 6) [255.         255.         255.           0.99999771   0.99999848
   0.99999987] [ 3.          3.          0.         -0.99999721 -0.99999923 -0.9999985 ]
[1. 1. 1. ... 1. 1. 1.]
(368473, 9)


In [6]:
'''
http://www.scan-net.org/ScanNet/

<scanId>
|-- <scanId>.sens
    RGB-D sensor stream containing color frames, depth frames, camera poses and other data
|-- <scanId>_vh_clean.ply
    High quality reconstructed mesh
|-- <scanId>_vh_clean_2.ply
    Cleaned and decimated mesh for semantic annotations
|-- <scanId>_vh_clean_2.0.010000.segs.json
    Over-segmentation of annotation mesh

|-- <scanId>.aggregation.json, <scanId>_vh_clean.aggregation.json
    Aggregated instance-level semantic annotations on lo-res, hi-res meshes, respectively
|-- <scanId>_vh_clean_2.0.010000.segs.json, <scanId>_vh_clean.segs.json
    Over-segmentation of lo-res, hi-res meshes, respectively (referenced by aggregated semantic annotations)
    
|-- <scanId>_vh_clean_2.labels.ply
    Visualization of aggregated semantic segmentation; colored by nyu40 labels (see img/legend; ply property 'label' denotes the nyu40 label id)
    
|-- <scanId>_2d-label.zip
    Raw 2d projections of aggregated annotation labels as 16-bit pngs with ScanNet label ids
|-- <scanId>_2d-instance.zip
    Raw 2d projections of aggregated annotation instances as 8-bit pngs
|-- <scanId>_2d-label-filt.zip
    Filtered 2d projections of aggregated annotation labels as 16-bit pngs with ScanNet label ids
|-- <scanId>_2d-instance-filt.zip
    Filtered 2d projections of aggregated annotation instances as 8-bit pngs
'''

mode = 'train'

# getting scene information
description_filepath = Path(
    filepath
).parent / filepath.name.replace("_vh_clean_2.ply", ".txt")
print(description_filepath)
with open(description_filepath) as f:
    scene_type = f.read().split("\n")[:-1]
scene_type = scene_type[-1].split(" = ")[1]
print(scene_type)

filebase["scene_type"] = scene_type
filebase["raw_description_filepath"] = description_filepath

# getting instance info
instance_info_filepath = next(
    Path(filepath).parent.glob("*.aggregation.json")
)
filebase["raw_instance_filepath"] = instance_info_filepath
print(instance_info_filepath)

segment_indexes_filepath = next(
    Path(filepath).parent.glob("*[0-9].segs.json")
)
print(segment_indexes_filepath)
segments = scannet_processor._read_json(segment_indexes_filepath)
segments = np.array(segments["segIndices"]) # // per-vertex index of mesh segment
filebase["raw_segmentation_filepath"] = segment_indexes_filepath

print(segments.shape, segments[0:5], np.unique(segments).shape)

# add segment id as additional feature
segment_ids = np.unique(segments, return_inverse=True)[1]
points = np.hstack((points, segment_ids[..., None]))
print(segment_ids.shape, segment_ids[:5], np.amax(segment_ids), np.amin(segment_ids))
print(points.shape) # (N, 9): x, y, z, R(255), G(255), B(255), nx, ny, nz, segment_ids

np.save(str('segment_ids.npy'), segment_ids)

/newfoundland/ScanNet/scans/scene0329_01/scene0329_01.txt
Lobby
/newfoundland/ScanNet/scans/scene0329_01/scene0329_01.aggregation.json
/newfoundland/ScanNet/scans/scene0329_01/scene0329_01_vh_clean_2.0.010000.segs.json
(368473,) [53442 53442 53442 53442 53442] (2817,)
(368473,) [526 526 526 526 526] 2816 0
(368473, 10)


In [5]:
print(np.amax(segments), np.amin(segments))
print(np.amax(segment_ids), np.amin(segment_ids))

95691 158
470 0


In [34]:
# reading labels file
'''
ply property 'label' denotes the nyu40 label id

https://github.com/ScanNet/ScanNet/blob/3e5726500896748521a6ceb81271b0f5b2c0e7d2/BenchmarkScripts/util.py#L81

[!!!] 0 - unannotated
'''

label_filepath = filepath.parent / filepath.name.replace(
    ".ply", ".labels.ply"
)
print(label_filepath)
filebase["raw_label_filepath"] = label_filepath
label_coords, label_colors, labels = load_ply_with_normals(
    label_filepath
)
if not np.allclose(coords, label_coords):
    raise ValueError("files doesn't have same coordinates")

print(label_coords.shape, np.amax(label_coords, axis=0), np.amin(label_coords, axis=0))
print(labels.shape, labels[:5], np.unique(labels), np.sum(labels == 0) / float(len(labels)))

/newfoundland/ScanNet/scans/scene0055_00/scene0055_00_vh_clean_2.labels.ply
(95715, 3) [3.7441688 3.1282358 2.2303934] [-0.03978582 -0.00638135  0.00121044]
(95715,) [2 2 8 8 8] [ 0  1  2  4  5  8  9 14 39] 0.06750248132476623


In [35]:
# adding instance label
labels = labels[:, np.newaxis]
empty_instance_label = np.full(labels.shape, -1) # [!!!!!] -1: empty instance label
labels = np.hstack((labels, empty_instance_label))
print(labels.shape, labels.dtype)

instance_db = scannet_processor._read_json(instance_info_filepath)
print(len(instance_db["segGroups"]), instance_db["segGroups"][0])

for instance in instance_db["segGroups"]:
    segments_occupied = np.array(instance["segments"])
    occupied_indices = np.isin(segments, segments_occupied)
    labels[occupied_indices, 1] = instance["id"]

    # if scannet_processor.scannet200:
    #     label200 = instance["label"]
    #     # Map the category name to id
    #     label_ids = scannet_processor.labels_pd[
    #         scannet_processor.labels_pd["raw_category"] == label200
    #     ]["id"]
    #     label_id = (
    #         int(label_ids.iloc[0]) if len(label_ids) > 0 else 0
    #     )
    #     labels[occupied_indices, 0] = label_id
    
print(points.shape, labels.shape)
points = np.hstack((points, labels))

# gt_data = (points[:, -2] + 1) * 1000 + points[:, -1] + 1
gt_data = points[:, -2] * 1000 + points[:, -1] + 1


(95715, 2) int64
13 {'id': 0, 'objectId': 0, 'segments': [8401, 6853, 9262, 15941, 9487, 8595, 9292, 4112, 9534, 8458, 9116, 4132, 8015, 5860, 9176, 6097, 6726, 6569, 8177, 7893, 7254, 7591, 8443, 5926, 6844, 8290, 8314, 6085, 9626, 6555, 9296, 14296, 6452, 8069, 9201], 'label': 'window'}
(95715, 12) (95715, 2)


In [44]:
print(np.amax(labels[:, -1]), np.amin(labels[:, -1]), np.unique(labels[:, -1]))

12 -1 [-1  0  1  2  3  4  5  6  7  8  9 10 11 12]


In [31]:
np.unique(labels[:, 1])

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])